In [32]:
from cavity_mode_expansion import CavityModeExpansion
import legume
import numpy as np

We setup the crystal cavity we are interested in simulating.

In [33]:
# Number of PhC periods in x and y directions
Nx, Ny = 12, 12


# Regular PhC parameters
a=1.0

dslab = 0.26/0.366
n_slab = 3.4
ra = 0.344


# Initialize a lattice and PhC
lattice = legume.Lattice([Nx*a, 0], [0, Ny*np.sqrt(3)/2*a])

# Make x and y positions in one quadrant of the supercell
# We only initialize one quadrant because we want to shift the holes symmetrically
xp, yp = [], []
nx, ny = Nx, Ny
for iy in range(ny):
    for ix in range(nx):
        if iy == 0 and ix in (-1 %nx,0,1):
            continue
        xp.append((ix + (iy%2)*0.5)*a)
        yp.append((iy*np.sqrt(3)/2)*a)

nc = len(xp)

we define some meta properties of the crystal.

In [34]:
super_periods = np.array((Nx*a, 2*Ny*np.sqrt(3)/2*a))
defect_margins = 0.25 * super_periods

# Define L3 PhC cavity with shifted holes
def cavity():
    # Initialize PhC
    phc = legume.PhotCryst(lattice)

    # Add a layer to the PhC
    phc.add_layer(d=dslab, eps_b=n_slab**2)

    # Apply holes symmetrically in the four quadrants
    for ic, x in enumerate(xp):
        yc = yp[ic]
        xc = xp[ic]

        phc.add_shape(legume.Circle(x_cent=xc, y_cent=yc, r=ra))
    return phc

phc = cavity()

Next, we construct the base lattice, the photonic crystal object, and the GME object:

In [35]:
a1 = np.array([a*0.5, a*np.sqrt(3)/2])
a2 = np.array([a*0.5, -a*np.sqrt(3)/2])

base_lattice = legume.Lattice(a1,a2)
base_phc = legume.PhotCryst(base_lattice)
base_phc.add_layer(d=dslab, eps_b=n_slab**2)
base_phc.layers[0].add_shape(legume.Circle(eps=1.0, r=ra))


We construct the cavity mode analysis object

In [36]:
cme = CavityModeExpansion(phc, base_phc, super_periods, defect_margins, layer=0, gmax=3, base_gmax=3)

First lets find the band-gaps of the base crystal. This will give us an idea of where the cavity modes should be.

In [37]:
band_gaps, gmg_ratio =cme.find_band_gaps(band_tol=0.04, order=np.array([0,3]), lc_tol=0.1)

# With malice of forethought we know there is only one band-gap for this crystal.
print("band gap lower bound: %f,\n"
      "band gap upper bound: %f,\n"
      "band gap middle: %f,\n"
      "gap-midgap ratio: %f." %  (band_gaps[0][0], band_gaps[0][1], band_gaps[0][2], gmg_ratio[0]))

band gap lower bound: 0.243921,
band gap upper bound: 0.286060,
band gap middle: 0.264990,
gap-midgap ratio: 0.159023.


From this data we can deduce that when we are looking for cavity modes we should be looking for modes in the region
$0.25 \frac{\omega a}{2\pi c}-0.38\frac{\omega a}{2\pi c}$. In an optimization we may want to use data like this
to tune the properties of the base crystal.

For example we may want to tune the thickness of the slab to rid ourselves of higher order modes.

Now that we have the frequency region we are interested in we can setup the simulation on
